# Understanding Reddit

In [1]:
import praw
from praw.models import MoreComments
import simplejson as json
from __future__ import print_function
import numpy as np
import string as s

## Initialize Reddit/Subreddit Instances

In [2]:
reddit = praw.Reddit(client_id='', 
                     client_secret='', 
                     user_agent='', 
                     username='', 
                     password='')

#### LeBron Joining the Lakers

In [32]:
# Instantiate the thread we're using 
submission = reddit.submission(url='https://www.reddit.com/r/nba/comments/8ve9rs/withers_lebron_signing_with_lakers/')
# Create empty list 
lebron = []
# Append top-level comments to list 
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    lebron.append(top_level_comment.body)
# The top comment is a stickied comment with no score, so remove it 
lebron = lebron[1:]

In [33]:
for i in range(3):
    print(lebron[i])
    print('\n')

as a lakers fan since july 2018 i can't put into words how much this means to me


Comments moving so fast no one will know I love my wife 


Does this hurt his chances of going to Houston?




In [58]:
# Determining number of upvotes
ids = []
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    ids.append(top_level_comment.id)
print(ids[:5])

[u'e1motzb', u'e1mpdfi', u'e1mosv8', u'e1moxl5', u'e1mojxq']


In [66]:
votes = []
for id_no in ids[1:10]: 
    comment = reddit.comment(id = id_no)
    votes.append(comment.score)

In [67]:
sum(votes)

113407

Tasks for Cleaning: 
    
1. Remove newlines. 
    
2. Remove urls. 

3. Make all words lowercase.

# Creating the Model

First, we'll train the model on the opinion-lexicon library. 

#### Necessary Imports and Formatting Function

In [1]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

from nltk.corpus import opinion_lexicon
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

import re

In [2]:
pos_words = list(opinion_lexicon.positive())
pos_words = [word.encode('ascii', 'ignore') for word in pos_words]
positive = []
for word in pos_words:
    positive.append(tuple([dict([[word, True]]), 'positive' ]))
    
neg_words = list(opinion_lexicon.negative())
neg_words = [word.encode('ascii', 'ignore') for word in neg_words]
negative = []
for word in neg_words:
    negative.append(tuple([dict([[word, True]]), 'negative']))

The following will determine how to train test split

In [65]:
print(round(len(positive)*.30))
print(round(len(negative)*.30))

602.0
1435.0


In [397]:
len(negative[1578:])

3205

In [4]:
train = positive[661:] + negative[1578:]
test = positive[:661] + negative[:1578]

In [5]:
classifier = NaiveBayesClassifier.train(train)
accuracy = nltk.classify.util.accuracy(classifier, test)
print(accuracy * 100)

70.47789191603394


Mistake! I didn't split the data properly. Keep in mind for the final report. 

# Implementing the Model

We still need to clean the data. We'll place all the steps within the format_words function. 

In [586]:
# Check if output would have been the same with a function 
def format_words(comments_input):
    # Create a copy of input list 
    new_comments = deepcopy(comments_input)
    
    for i in range(len(new_comments)): 
        # First, change coding from unicode to string 
        new_comments[i] = new_comments[i].encode('ascii', 'ignore')
        # Remove newlines, punctuation, and urls 
        new_comments[i] = re.sub(r'\n', ' ', new_comments[i])
        new_comments[i] = re.sub(r'[^\w\s]', '', new_comments[i])
        new_comments[i] = re.sub(r'http\S+', '', new_comments[i])
        # Make every word lowercase 
        new_comments[i] = new_comments[i].lower()
    
    words = []
    for comment in new_comments:
        tokenized = word_tokenize(comment)
        # Remove stopwords
        useful_words = [word for word in tokenized if word not in stopwords.words('english')]
        # Create dictionary
        my_dict = dict([(word, True) for word in useful_words])
        # Append to words
        words.append(my_dict)
    # Return final list
    return(words)

In [588]:
input_data = format_words(lebron)
pos = 0
neg = 0
for tweet in input_data:
    result = classifier.classify(tweet)
    if result == 'negative':
        neg = neg + 1
    if result == 'positive':
        pos = pos + 1

In [591]:
print('About {}% of comments regarding LeBron\'s move were negative.'.format(round((float(neg)/(neg+pos)*100), 2)))

About 59.82% of comments regarding LeBron's move were negative.
